In [23]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

In [24]:
model = YOLO('../best.pt')

In [25]:
imgs = []

In [26]:
filenames = sorted(os.listdir('imgs_test'), key=lambda x: int(x.split('.')[0]))

In [27]:
# Load each image, convert it to a byte array, and append to the list
for filename in filenames:
    with open(f'imgs_test/{filename}', 'rb') as f:
        imgs.append(f.read())

In [28]:
# Function to convert byte data to an image
def bytes_to_image(byte_data):
    np_arr = np.frombuffer(byte_data, np.uint8)
    image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    print(type(image))
    return image


In [47]:
image.shape

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 45,  45,  45],
        [ 50,  50,  50],
        [ 51,  51,  51]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 44,  44,  44],
        [ 48,  48,  48],
        [ 49,  49,  49]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 41,  41,  41],
        [ 47,  47,  47],
        [ 49,  49,  49]]

In [43]:
np_arr = np.frombuffer(imgs[0], np.uint8)

In [ ]:
# Define the entry/exit boundary line (e.g., a horizontal line at y=250)
boundary_y = 50
entry_count = 0
exit_count = 0

In [ ]:
confidence_threshold = 0.9

In [ ]:
previous_positions = {}

In [ ]:
def process_and_track(frames):
    global entry_count, exit_count, previous_positions
    
    for byte_data in frames:
        # Convert byte data to image
        frame = bytes_to_image(byte_data)
        
        # Ensure the frame was converted successfully
        if frame is None:
            continue
        
        # Track objects in the frame
        results = model.track(source=frame)
        
        # Process the tracking results (e.g., draw bounding boxes)
        current_positions = {}
        
        for result in results:
            for box in result.boxes:
                confidence = box.conf[0].item()

                if confidence < confidence_threshold:
                    continue

                x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
                object_id = int(box.id[0])  # Assuming 'id' contains the unique ID
                
                # Calculate the center of the bounding box
                center_y = (y1 + y2) // 2
                
                # Store the current position
                current_positions[object_id] = center_y
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                #cv2.putText(frame, f'ID: {object_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                #cv2.putText(frame, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Check for entries and exits
        for object_id, center_y in current_positions.items():
            if object_id in previous_positions:
                previous_y = previous_positions[object_id]
                
                # Check if the object has crossed the boundary
                if previous_y <= boundary_y < center_y:
                    entry_count += 1
                elif previous_y >= boundary_y > center_y:
                    exit_count += 1
        
        # Update previous positions
        previous_positions = current_positions
        
        # Draw the boundary line
        cv2.line(frame, (0, boundary_y), (frame.shape[1], boundary_y), (0, 255, 0), 2)
        
        # Display the counts with smaller font size
        cv2.putText(frame, f'Entries: {entry_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.putText(frame, f'Exits: {exit_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
        # Resize the frame to 500x500 pixels
        resized_frame = cv2.resize(frame, (500, 500))
        
        # Display the frame (optional, for visualization purposes)
        cv2.imshow('Tracked Frame', resized_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [ ]:
process_and_track(imgs)
cv2.destroyAllWindows()